# Mandelbrot Set

## Background

The Mandelbrot Set is a set of values in the complex plane where the function $z_{n+1} = z_n^2 + c$ does not diverge when iterated from $z = 0$. A complex number $c$ is a member of the Mandelbrot set if the absolute value of $z_n$ remains bounded with repeated iterations starting from $z_0 = 0$.

The cell below shows a plot of the Mandelbrot set. Points coloured in black belong to the set, and all other points do not.

<img src="images/mandel.png" style="width:300px;height:250px"/>

## Software Version

The code below draws the mandelbrot set using Python's matplotlib library. The code executes quite slowly, especially the third cell where the function mandelbrot is called for each point of the plotted image. This is due to the very large amount of calculations performed, as calculations are done for many iterations over each pixel of the image.

This first cell imports the matplotlib and numpy libraries.

In [ ]:
import matplotlib.pyplot as plt
import numpy
from ipywidgets import *

The second cell defines the function mandelbrot.

In [ ]:
def mandelbrot(Re,Im,max_iter):
    c = complex(Re,Im)
    z = complex(0,0)
    for i in range(max_iter):
        z = z*z + c
        if (abs(z) > 2):
            return i
    return max_iter

The third cell creates the plot for the Mandelbrot Set.

The numbers of iterations that will be run for each pixel can be adjusted using a slider widget. This ranges from 5 to 50 to allow the user to see the effect of altering the number of iterations.

Set the iteration value using the slider and then run by pressing the button.

In [ ]:
def update(Iterations):
    cols = 200
    rows = 200
    result = numpy.zeros([rows,cols])
    for row_index, Re in enumerate(numpy.linspace(-2,1,num=rows)):
        for col_index, Im in enumerate(numpy.linspace(-1,1,num=cols)):
            result[row_index,col_index] = mandelbrot(Re,Im,Iterations)
    plt.figure(dpi=150)
    plt.imshow(result.T,cmap="jet",extent=[-2,1,-1,1])
    plt.xlabel("Re")
    plt.ylabel("Im")
    plt.show()

ite = widgets.IntSlider(min=5,max=50,step=1,value=25)
interact_manual(update, Iterations=ite);

The above image is the plot of the Mandelbrot set. The red area represents complex numbers that are in the Mandelbrot set, and all other colours represent complex numbers not in the set.

## Hardware Version

Some of the more mathematically intensive processes will be "off-loaded" to the Programmable Logic (PL) of the Pynq Z2 board. This will be done by creating Intellectual Property (IP) cores to perform the specific tasks. IP cores were created to do the tasks in lines 5 and 6 of the second cell. One IP core was created to approximate the magnitude of a complex number.  The second IP core is designed to find the square of a complex number. 

### Magnitude IP Core

The IP core for finding the magnitude of a complex number is done using the Alpha max plus beta min algorithm [1]. This does not give a completely accurate result, but it is a fast operation that can be performed easily on FPGA hardware. The algorithm is faster as it avoids performing squaring or square root operations. The approximation is:

<img src="images/abEquation.png" style="width:350px;height:60px"/>

Where Max is the maximum absolute value of a and b, and Min is the minimum absolute value of a and b.

The IP core design is shown below.

<img src="images/magnitude.png"/>

The IP core was designed in System Generator as the block diagram approach makes it easy to see and understand the flow of the design. The design takes in two inputs, for the Real and Imaginary parts of the complex number, respectively. A Relational block is used to compare the two inputs, and returns a boolean true or false, based on whether or not the real input is greater than the imaginary input. The output of this block is fed into two multiplexers which select the maximum and minimum of the two values, respectively. 
The maximum value is then multiplied by Alpha, and the minimum by Beta. The optimum values of Alpha and Beta for this algorithm are as shown below.

<img src="images/alphabeta.png" style="width:250px;height:100px"/>

These values are used in the CMult blocks, and have been rounded to 4 decimal places. This will reduce the wordlength required to represent them and therefore will minimise FPGA resources required. The results of the two CMult blocks are then summed, giving the final result for the approximation of the magnitude.

This example takes the input $(1 + i)$ and gives the approximation 1.358. The exact value of magnitude is 1.414, meaning the margin of error in the approximation is 3.96%. This error is acceptable for our program, as we only need to check if the magnitude is greater than 2.

### Complex Square IP Core

The IP core design for finding the square of a complex number is shown below.

<img src="images/compSquare.png"/>

The design was derived by considering the square of a complex number in the form $(A + iB)$. Squaring this complex number gives the result $(A^{2} + B^{2}) +i(2AB)$. The design implements this equation to give the real and imaginary part of the complex square, where the real part is $(A^{2} + B^{2})$ and the imaginary part is $2AB$.

### IP Core Generation

The two previously shown designs were added as IP cores to a custom overlay in Vivado. The block diagram for the overlay is shown below.

<img src="images/blockdiagram.png"/>

This shows the two IP cores, compsquare and magnitude, as part of the overlay.

After validating the design, the HDL wrapper was created in Vivado, before creating the bitstream (.bit), hardware hand-off (.hwh) and Tcl files. These three files were uploaded to the storage of the PYNQ board for use in this project.

### Program

The below cell imports the Overlay class from the pynq library, and then creates a new overlay and parses the design in order to understand the contents of the overlay. Also imported here are the matplotlib and numpy libraries for Python.

In [ ]:
from pynq import Overlay
ol = Overlay("design_mandel_wrapper.bit")
import matplotlib.pyplot as plt
import numpy
from ipywidgets import *

The cell below returns a report of the overlay. When run, it shows both IP cores under the "IP Blocks" heading.

In [ ]:
ol?

The cell below gives the IP cores a simpler alias so they can be more easily referenced.

In [ ]:
compSq = ol.compsquare_0
mag = ol.magnitude_0

This cell defines functions to make it easier to use the IP cores, as well as a function for converting unsigned numbers to signed.

In [ ]:
def to_signed(val,b):
    signedVal = val-(2**b)*int(str((val)>>b-1))
    return signedVal

def square(z):
    compSq.write(0x00, int((z.real*2**(16)))) #real input
    compSq.write(0x04, int((z.imag*2**(16)))) #imaginary input
    re = to_signed(compSq.read(0x0C),32)*2**(-16)
    im = to_signed(compSq.read(0x08),32)*2**(-16)
    sq = complex(re,im) #output
    return sq

def magnitude(z):
    mag.write(0x00, abs(int((z.real*2**(16))))) #real input
    mag.write(0x04, abs(int((z.imag*2**(16)))))#imaginary input
    magnitude = ((mag.read(0x08))*2**(-16))
    return magnitude

The writes to the IP cores are scaled by a factor of $2^{16}$ to allow for the fractional part of the answer.
Similarly, the values read from the IP core are scaled by a factor of $2^{-16}$.

The cell below defines the mandelbrot function using the functions of the IP cores.

In [ ]:
def mandelbrotPL(Re,Im,max_iter):
    c = complex(Re,Im)
    z = complex(0,0)
    for i in range(max_iter):
        z = square(z) + c
        if (magnitude(z) > 2):
            return i
    return max_iter

The third cell creates the plot for the Mandelbrot Set.

The numbers of iterations that will be run for each pixel can be adjusted using a slider widget. This ranges from 5 to 50 to allow the user to see the effect of altering the number of iterations. 

Set the iteration value using the slider and then run by pressing the button. Note that higher numbers of iterations will take significantly longer to run.

In [ ]:
def updatePL(Iterations):
    cols = 200
    rows = 200
    result = numpy.zeros([rows,cols])
    for row_index, Re in enumerate(numpy.linspace(-2,1,num=rows)):
        for col_index, Im in enumerate(numpy.linspace(-1,1,num=cols)):
            result[row_index,col_index] = mandelbrotPL(Re,Im,Iterations)
    plt.figure(dpi=150)
    plt.imshow(result.T,cmap="jet",extent=[-2,1,-1,1])
    plt.xlabel("Re")
    plt.ylabel("Im")
    plt.show()

ite = widgets.IntSlider(min=5,max=50,step=1,value=25)
interact_manual(updatePL, Iterations=ite);

The above image shows the plot for the Mandelbrot set as created using the PYNQ board's programmable logic.

It can be seen that the above version of the code runs much slower than the code running all on the Processing System of the PYNQ board. This is due to the AXI-Lite interface used to go between the PL and PS. The additional delay is not noticeable in smaller programs run only once, however, as this program involves many iterations of the same function, the time adds up to a large delay. This could be improved in further work by parallelising the design so that it works on multiple pixels at any one time, or by implementing the AXI stream interface. 

## References

[1] Lyons, Richard G. Understanding Digital Signal Processing, section 13.2. Prentice Hall, 2004 ISBN 0-13-108989-7.